# Amazon SageMaker Multi-Model Endpoints using Linear Learner
With [Amazon SageMaker multi-model endpoints](https://docs.aws.amazon.com/sagemaker/latest/dg/multi-model-endpoints.html), customers can create an endpoint that seamlessly hosts up to thousands of models. These endpoints are well suited to use cases where any one of a large number of models, which can be served from a common inference container, needs to be invokable on-demand and where it is acceptable for infrequently invoked models to incur some additional latency. For applications which require consistently low inference latency, a traditional endpoint is still the best choice.

At a high level, Amazon SageMaker manages the loading and unloading of models for a multi-model endpoint, as they are needed. When an invocation request is made for a particular model, Amazon SageMaker routes the request to an instance assigned to that model, downloads the model artifacts from S3 onto that instance, and initiates loading of the model into the memory of the container. As soon as the loading is complete, Amazon SageMaker performs the requested invocation and returns the result. If the model is already loaded in memory on the selected instance, the downloading and loading steps are skipped and the invocation is performed immediately.

Amazon SageMaker inference pipeline model consists of a sequence of containers that serve inference requests by combining preprocessing, predictions and post-processing data science tasks.  An inference pipeline allows you to apply the same preprocessing code used during model training, to process the inference request data used for predictions.

To demonstrate how multi-model endpoints are created and used with inference pipeline, this notebook provides an example using a set of Linear Learner models that each predict housing prices for a single location. This domain is used as a simple example to easily experiment with multi-model endpoints.  

This notebook showcases three MME capabilities: 
* Native MME support with Amazon SageMaker Linear Learner algorithm.  Because of the native support there is no need for you to create a custom container.  
* Native MME support with Amazon SageMaker Inference Pipelines.
* Granular InvokeModel access to multiple models hosted on the MME using IAM condition key.

To demonstrate these capabilities, the notebook discusses the use case of predicting house prices in multiple cities using linear regression.  House prices are predicted based on features like number of bedrooms, number of garages, square footage etc.  Depending on the city, the features affect the house price differently.  For example, small changes in the square footage cause a drastic change in house prices in New York when compared to price changes in Houston.  For accurate house price predictions, we will train multiple linear regression models, a unique location specific model per city.  


### Contents

1. [Generate synthetic data for housing models](#Generate-synthetic-data-for-housing-models)
1. [Preprocess the raw housing data using Scikit Learn model](#Preprocess-synthetic-housing-data-using-scikit-learn)
1. [Train multiple house value prediction models for multiple cities](#Train-multiple-house-value-prediction-models)
1. [Create model entity with multi model support](#Create-sagemaker-multi-model-support)
1. [Create an inference pipeline with sklearn model and MME linear learner model](#Create-inference-pipeline)
1. [Exercise the inference pipeline - Get predictions from the different  linear learner models](#Exercise-inference-pipeline)
1. [Update Multi Model Endpoint with new models](#update-models)
1. [Explore granular access to the target models of MME](#Finegrain-control-invoke-models)
1. [Endpoint CloudWatch Metrics Analysis](#CW-metric-analysis)
1. [Clean up](#CleanUp)


## Section 1 - Generate synthetic data for housing models <a id='Generate-synthetic-data-for-housing-models'></a>

In this section, you will generate synthetic data that will be used to train the linear learner models.  The data generated consists of 6 numerical features - the year the house was built in, house size in square feet, number of bedrooms, number of bathroom, the lot size and number of garages and two categorial features - deck and front_porch.  

In [1]:
import numpy as np
import pandas as pd
import json
import datetime
import time
import boto3
import sagemaker
import os

from time import gmtime, strftime
from random import choice

from sagemaker import get_execution_role

from sagemaker.multidatamodel import MULTI_MODEL_CONTAINER_MODE
from sagemaker.multidatamodel import MultiDataModel

from sklearn.model_selection import train_test_split

In [2]:
NUM_HOUSES_PER_LOCATION = 1000
LOCATIONS  = ['NewYork_NY',    'LosAngeles_CA',   'Chicago_IL',    'Houston_TX',   'Dallas_TX',
              'Phoenix_AZ',    'Philadelphia_PA', 'SanAntonio_TX', 'SanDiego_CA',  'SanFrancisco_CA']
MAX_YEAR = 2019

In [3]:
def gen_price(house):
    """Generate price based on features of the house"""
    
    if house['FRONT_PORCH'] == 'y':
        garage = 1
    else:
        garage = 0
        
    if house['FRONT_PORCH'] == 'y':
        front_porch = 1
    else:
        front_porch = 0
        
    price = int(150 * house['SQUARE_FEET'] + \
                10000 * house['NUM_BEDROOMS'] + \
                15000 * house['NUM_BATHROOMS'] + \
                15000 * house['LOT_ACRES'] + \
                10000 * garage + \
                10000 * front_porch + \
                15000 * house['GARAGE_SPACES'] - \
                5000 * (MAX_YEAR - house['YEAR_BUILT']))
    return price

In [4]:
def gen_yes_no():
    """Generate values (y/n) for categorical features"""
    answer = choice(['y', 'n'])
    return answer

In [5]:
def gen_random_house():
    """Generate a row of data (single house information)"""
    house = {'SQUARE_FEET':    np.random.normal(3000, 750),
             'NUM_BEDROOMS':  np.random.randint(2, 7),
             'NUM_BATHROOMS': np.random.randint(2, 7) / 2,
             'LOT_ACRES':     round(np.random.normal(1.0, 0.25), 2),
             'GARAGE_SPACES': np.random.randint(0, 4),
             'YEAR_BUILT':    min(MAX_YEAR, int(np.random.normal(1995, 10))),
             'FRONT_PORCH':   gen_yes_no(),
             'DECK':          gen_yes_no()
            }
    
    price = gen_price(house)
    
    return [house['YEAR_BUILT'],   
            house['SQUARE_FEET'], 
            house['NUM_BEDROOMS'], 
            house['NUM_BATHROOMS'], 
            house['LOT_ACRES'],    
            house['GARAGE_SPACES'],
            house['FRONT_PORCH'],    
            house['DECK'], 
            price]

In [6]:
def gen_houses(num_houses):
    """Generate housing dataset"""
    house_list = []
    
    for _ in range(num_houses):
        house_list.append(gen_random_house())
        
    df = pd.DataFrame(
        house_list, 
        columns=[
            'YEAR_BUILT',    
            'SQUARE_FEET',  
            'NUM_BEDROOMS',            
            'NUM_BATHROOMS',
            'LOT_ACRES',
            'GARAGE_SPACES',
            'FRONT_PORCH',
            'DECK', 
            'PRICE']
    )
    return df

In [7]:
def save_data_locally(location, train, test): 
    """Save the housing data locally"""
    os.makedirs('data/{0}/train'.format(location), exist_ok=True)
    train.to_csv('data/{0}/train/train.csv'.format(location), sep=',', header=False, index=False)
       
    os.makedirs('data/{0}/test'.format(location), exist_ok=True)
    test.to_csv('data/{0}/test/test.csv'.format(location), sep=',', header=False, index=False) 

In [8]:
#Generate housing data for multiple locations.
#Change "PARALLEL_TRAINING_JOBS " to a lower number to limit the number of training jobs and models. Or to a higher value to experiment with more models.

PARALLEL_TRAINING_JOBS = 4

for loc in LOCATIONS[:PARALLEL_TRAINING_JOBS]:
    houses = gen_houses(NUM_HOUSES_PER_LOCATION)
    
    #Spliting data into train and test in 90:10 ratio
    #Not splitting the train data into train and val because its not preprocessed yet
    train, test = train_test_split(houses, test_size=0.1)
    save_data_locally(loc, train, test)


In [9]:
#Shows the first few lines of data.
houses.head()

,YEAR_BUILT,SQUARE_FEET,NUM_BEDROOMS,NUM_BATHROOMS,LOT_ACRES,GARAGE_SPACES,FRONT_PORCH,DECK,PRICE
0,2014,3236.316691,6,1.5,1.13,3,y,n,624897
1,1991,2167.509634,4,2.0,1.39,0,n,y,275976
2,2016,1599.113490,3,1.0,1.20,1,n,n,302867
3,2016,2772.308254,2,1.5,0.98,3,y,n,523046
4,2005,3099.231612,6,2.5,1.19,0,n,n,510234


## Section 2 - Preprocess the raw housing data using Scikit Learn <a id='Preprocess-synthetic-housing-data-using-scikit-learn'></a>

In this section, the categorical features of the data (deck and porch) are pre-processed using sklearn to convert them to one hot encoding representation.  

In [10]:
sm_client = boto3.client(service_name='sagemaker')
runtime_sm_client = boto3.client(service_name='sagemaker-runtime')
sagemaker_session = sagemaker.Session()

s3 = boto3.resource('s3')
s3_client = boto3.client('s3')

BUCKET  = sagemaker_session.default_bucket()
print("BUCKET : ", BUCKET)

role = get_execution_role()
print("ROLE : ", role)

ACCOUNT_ID = boto3.client('sts').get_caller_identity()['Account']
REGION = boto3.Session().region_name

DATA_PREFIX = 'DEMO_MME_LINEAR_LEARNER'
HOUSING_MODEL_NAME = 'housing'
MULTI_MODEL_ARTIFACTS = 'multi_model_artifacts'

BUCKET :  sagemaker-us-west-2-688520471316
ROLE :  arn:aws:iam::688520471316:role/hongshan-sagemaker-experiment


In [11]:
#Create the SKLearn estimator with the sklearn_preprocessor.py as the script
from sagemaker.sklearn.estimator import SKLearn

script_path = 'sklearn_preprocessor.py'

sklearn_preprocessor = SKLearn(
    entry_point=script_path,
    role=role,
    instance_type="ml.c4.xlarge",
    framework_version="0.20.0",
    sagemaker_session=sagemaker_session)

In [12]:
#Upload the raw training data to S3 bucket, to be accessed by SKLearn
train_inputs = []

for loc in LOCATIONS[:PARALLEL_TRAINING_JOBS]:

    train_input = sagemaker_session.upload_data(
        path='data/{}/train/train.csv'.format(loc),
        bucket=BUCKET,
        key_prefix='housing-data/{}/train'.format(loc)
    )
    
    train_inputs.append(train_input)
    print("Raw training data uploaded to : ", train_input)

Raw training data uploaded to :  s3://sagemaker-us-west-2-688520471316/housing-data/NewYork_NY/train/train.csv
Raw training data uploaded to :  s3://sagemaker-us-west-2-688520471316/housing-data/LosAngeles_CA/train/train.csv
Raw training data uploaded to :  s3://sagemaker-us-west-2-688520471316/housing-data/Chicago_IL/train/train.csv
Raw training data uploaded to :  s3://sagemaker-us-west-2-688520471316/housing-data/Houston_TX/train/train.csv


In [13]:
##Launch multiple scikit learn training to process the raw synthetic data generated for multiple locations.
##Before executing this, take the training instance limits in your account and cost into consideration.

sklearn_preprocessors = []
sklearn_preprocessors_preprocessor_jobs = []

for index, loc in enumerate(LOCATIONS[:PARALLEL_TRAINING_JOBS]):
    print("preprocessing fit input data at ", index , " for loc ", loc)
     
    job_name='scikit-learn-preprocessor-{}'.format(strftime('%Y-%m-%d-%H-%M-%S', gmtime()))
    
    sklearn_preprocessor.fit({'train': train_inputs[index]}, job_name=job_name, wait=False)

    sklearn_preprocessors.append(sklearn_preprocessor)
    sklearn_preprocessors_preprocessor_jobs.append(job_name)
    
    time.sleep(1)

preprocessing fit input data at  0  for loc  NewYork_NY
preprocessing fit input data at  1  for loc  LosAngeles_CA
preprocessing fit input data at  2  for loc  Chicago_IL
preprocessing fit input data at  3  for loc  Houston_TX


In [14]:
def wait_for_training_job_to_complete(job_name):
    """ Wait for the training job to complete """
    print('Waiting for job {} to complete...'.format(job_name))
    
    waiter = sm_client.get_waiter('training_job_completed_or_stopped')
    waiter.wait(TrainingJobName=job_name)

In [15]:
def wait_for_batch_transform_job_to_complete(job_name):
    """Wait for the batch transform job to complete"""
    print('Waiting for job {} to complete...'.format(job_name))
    
    waiter = sm_client.get_waiter('transform_job_completed_or_stopped')
    waiter.wait(TransformJobName=job_name)

In [16]:
#Wait for the preprocessor jobs to finish
for job_name in sklearn_preprocessors_preprocessor_jobs:
    wait_for_training_job_to_complete(job_name)

Waiting for job scikit-learn-preprocessor-2021-05-28-20-30-42 to complete...
Waiting for job scikit-learn-preprocessor-2021-05-28-20-30-43 to complete...
Waiting for job scikit-learn-preprocessor-2021-05-28-20-30-45 to complete...
Waiting for job scikit-learn-preprocessor-2021-05-28-20-30-46 to complete...


In [17]:
##Once the preprocessor is fit, use tranformer to preprocess the raw training data and store the transformed data right back into s3.
##Before executing this, take the training instance limits in your account and cost into consideration.

preprocessor_transformers = []

for index, loc in enumerate(LOCATIONS[:PARALLEL_TRAINING_JOBS]):
    print("Transform the raw data at ", index , " for loc ", loc)
       
    sklearn_preprocessor = sklearn_preprocessors[index]
    
    transformer = sklearn_preprocessor.transformer(
        instance_count=1,
        instance_type='ml.m4.xlarge',
        assemble_with='Line',
        accept='text/csv'
    )
    
    preprocessor_transformers.append(transformer)

Transform the raw data at  0  for loc  NewYork_NY
Transform the raw data at  1  for loc  LosAngeles_CA
Transform the raw data at  2  for loc  Chicago_IL
Transform the raw data at  3  for loc  Houston_TX


In [18]:
# Preprocess training input
preprocessed_train_data_path = []

for index, transformer in enumerate(preprocessor_transformers):
    transformer.transform(train_inputs[index], content_type='text/csv')
    print('Launching batch transform job: {}'.format(transformer.latest_transform_job.job_name))
    preprocessed_train_data_path.append(transformer.output_path)

.................................
Processing /opt/ml/code
  Created wheel for sklearn-preprocessor: filename=sklearn_preprocessor-1.0.0-py2.py3-none-any.whl size=7511 sha256=e00ab46063caf72697d42e64febe16e009c8ee342dfd25b706671b7d1b24e3c7
  Stored in directory: /tmp/pip-ephem-wheel-cache-j3axvnb1/wheels/3e/0f/51/2f1df833dd0412c1bc2f5ee56baac195b5be563353d111dca6
Successfully built sklearn-preprocessor
/miniconda3/lib/python3.7/site-packages/sklearn/externals/joblib/externals/cloudpickle/cloudpickle.py:47: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp
[2021-05-28 20:40:19 +0000] [36] [INFO] Starting gunicorn 20.0.4
[2021-05-28 20:40:19 +0000] [36] [INFO] Listening at: unix:/tmp/gunicorn.sock (36)
[2021-05-28 20:40:19 +0000] [36] [INFO] Using worker: gevent
[2021-05-28 20:40:19 +0000] [39] [INFO] Booting worker with pid: 39
[2021-05-28 20:40:19 +0000] [40] [INFO] Booting worker with pid: 40
[2021-

In [19]:
#Wait for all the batch transform jobs to finish
for transformer in preprocessor_transformers: 
    job_name=transformer.latest_transform_job.job_name
    wait_for_batch_transform_job_to_complete(job_name)

Waiting for job sagemaker-scikit-learn-2021-05-28-20-34-55-699 to complete...
Waiting for job sagemaker-scikit-learn-2021-05-28-20-40-39-564 to complete...
Waiting for job sagemaker-scikit-learn-2021-05-28-20-45-52-957 to complete...
Waiting for job sagemaker-scikit-learn-2021-05-28-20-51-36-648 to complete...


In [20]:
##Download the preprocessed data, split into train and val, upload back to S3 in the same directory as tranformer output path
for index, transformer in enumerate(preprocessor_transformers): 
    transformer_output_key='{}/{}'.format(transformer.latest_transform_job.job_name, 'train.csv.out') 
    
    preprocessed_data_download_dir = '{}/'.format("preprocessed-data/"+LOCATIONS[index])
    
    sagemaker_session.download_data(
        path=preprocessed_data_download_dir, 
        bucket=BUCKET,
        key_prefix=transformer_output_key
    )
    
    print('transformer_output_key: {}'.format(transformer_output_key ))
    print('Download directory: {}'.format(preprocessed_data_download_dir ))
    
    train_df = pd.read_csv('{}/{}'.format(preprocessed_data_download_dir,"train.csv.out"))
    
    #Spliting data into train and test in 70:30 ratio
    train, val = train_test_split(train_df, test_size=0.3)
    
    train.to_csv('{}{}'.format(preprocessed_data_download_dir,"train.csv"), sep=',', header=False, index=False)
    val.to_csv('{}{}'.format(preprocessed_data_download_dir,"val.csv"), sep=',', header=False, index=False)
    
    
    train_input = sagemaker_session.upload_data(
        path='{}/{}'.format(preprocessed_data_download_dir, 'train.csv'), 
        bucket=BUCKET,
        key_prefix='{}'.format(transformer.latest_transform_job.job_name, 'train.csv'))
    
    val_input = sagemaker_session.upload_data(
        path='{}/{}'.format(preprocessed_data_download_dir, 'val.csv'), 
        bucket=BUCKET,
        key_prefix='{}'.format(transformer.latest_transform_job.job_name, 'val.csv'))

transformer_output_key: sagemaker-scikit-learn-2021-05-28-20-34-55-699/train.csv.out
Download directory: preprocessed-data/NewYork_NY/
transformer_output_key: sagemaker-scikit-learn-2021-05-28-20-40-39-564/train.csv.out
Download directory: preprocessed-data/LosAngeles_CA/
transformer_output_key: sagemaker-scikit-learn-2021-05-28-20-45-52-957/train.csv.out
Download directory: preprocessed-data/Chicago_IL/
transformer_output_key: sagemaker-scikit-learn-2021-05-28-20-51-36-648/train.csv.out
Download directory: preprocessed-data/Houston_TX/


In [21]:
##S3 location of the preprocessed data
for preprocessed_train_data in preprocessed_train_data_path: 
    print(preprocessed_train_data)

s3://sagemaker-us-west-2-688520471316/sagemaker-scikit-learn-2021-05-28-20-34-55-699
s3://sagemaker-us-west-2-688520471316/sagemaker-scikit-learn-2021-05-28-20-40-39-564
s3://sagemaker-us-west-2-688520471316/sagemaker-scikit-learn-2021-05-28-20-45-52-957
s3://sagemaker-us-west-2-688520471316/sagemaker-scikit-learn-2021-05-28-20-51-36-648


In [22]:
for index, loc in enumerate(LOCATIONS[:PARALLEL_TRAINING_JOBS]):
    preprocessed_data_download_dir = '{}/'.format("preprocessed-data/"+LOCATIONS[index])
    path='{}/{}'.format(preprocessed_data_download_dir, 'train.csv')

## Section 3 : Train house value prediction models for multiple cities <a id='Train-multiple-house-value-prediction-models'></a>

In this section, you will use the preprocessed housing data to train multiple linear learner models.

In [23]:
container = sagemaker.image_uris.retrieve(region=boto3.Session().region_name, framework='linear-learner')

### Launch a single training job for a given housing location
There is nothing specific to multi-model endpoints in terms of the models it will host. They are trained in the same way as all other SageMaker models. Here we are using the Linear Learner estimator and not waiting for the job to complete.

In [24]:
def launch_training_job(location, transformer):
    """Launch a linear learner traing job"""
    
    train_inputs = '{}/{}'.format(transformer.output_path, "train.csv")
    val_inputs = '{}/{}'.format(transformer.output_path, "val.csv")
    
    print("train_inputs:", train_inputs)
    print("val_inputs:", val_inputs)
     
    full_output_prefix = '{}/model_artifacts/{}'.format(DATA_PREFIX, location)
    s3_output_path = 's3://{}/{}'.format(BUCKET, full_output_prefix)
    
    print("s3_output_path ", s3_output_path)
    
    s3_output_path = 's3://{}/{}/model_artifacts/{}'.format(BUCKET, DATA_PREFIX, location)
    
    linear_estimator = sagemaker.estimator.Estimator(
                            container,
                            role, 
                            instance_count=1,
                            instance_type='ml.c4.xlarge',
                            output_path=s3_output_path,
                            sagemaker_session=sagemaker_session)
    
    linear_estimator.set_hyperparameters(
                           feature_dim=10,
                           mini_batch_size=100,
                           predictor_type='regressor',
                           epochs=10,
                           num_models=32,
                           loss='absolute_loss')
    
    DISTRIBUTION_MODE = 'FullyReplicated'
    train_input = sagemaker.inputs.TrainingInput(s3_data=train_inputs,
                                     distribution=DISTRIBUTION_MODE, content_type='text/csv;label_size=1')
    val_input   = sagemaker.inputs.TrainingInput(s3_data=val_inputs,
                                     distribution=DISTRIBUTION_MODE, content_type='text/csv;label_size=1')
    
    remote_inputs = {'train': train_input, 'validation': val_input}
     
    linear_estimator.fit(remote_inputs, wait=False)
   
    return linear_estimator.latest_training_job.name

### Kick off a model training job for each housing location

In [25]:

training_jobs = []
    
for transformer, loc in zip(preprocessor_transformers, LOCATIONS[:PARALLEL_TRAINING_JOBS]): 
    job = launch_training_job(loc, transformer)
    training_jobs.append(job)
    
print('{} training jobs launched: {}'.format(len(training_jobs), training_jobs))

train_inputs: s3://sagemaker-us-west-2-688520471316/sagemaker-scikit-learn-2021-05-28-20-34-55-699/train.csv
val_inputs: s3://sagemaker-us-west-2-688520471316/sagemaker-scikit-learn-2021-05-28-20-34-55-699/val.csv
s3_output_path  s3://sagemaker-us-west-2-688520471316/DEMO_MME_LINEAR_LEARNER/model_artifacts/NewYork_NY
train_inputs: s3://sagemaker-us-west-2-688520471316/sagemaker-scikit-learn-2021-05-28-20-40-39-564/train.csv
val_inputs: s3://sagemaker-us-west-2-688520471316/sagemaker-scikit-learn-2021-05-28-20-40-39-564/val.csv
s3_output_path  s3://sagemaker-us-west-2-688520471316/DEMO_MME_LINEAR_LEARNER/model_artifacts/LosAngeles_CA
train_inputs: s3://sagemaker-us-west-2-688520471316/sagemaker-scikit-learn-2021-05-28-20-45-52-957/train.csv
val_inputs: s3://sagemaker-us-west-2-688520471316/sagemaker-scikit-learn-2021-05-28-20-45-52-957/val.csv
s3_output_path  s3://sagemaker-us-west-2-688520471316/DEMO_MME_LINEAR_LEARNER/model_artifacts/Chicago_IL
train_inputs: s3://sagemaker-us-west-2-6

### Wait for all  training jobs to finish

In [26]:
#Wait for the jobs to finish
for job_name in training_jobs:
    wait_for_training_job_to_complete(job_name)

Waiting for job linear-learner-2021-05-28-20-56-51-295 to complete...
Waiting for job linear-learner-2021-05-28-20-56-51-472 to complete...
Waiting for job linear-learner-2021-05-28-20-56-53-549 to complete...
Waiting for job linear-learner-2021-05-28-20-56-58-082 to complete...


## Section 4 - Create Sagemaker model with multi model support <a id='Create-sagemaker-multi-model-support'></a>

In [27]:
import re
def parse_model_artifacts(model_data_url):
    # extract the s3 key from the full url to the model artifacts
    s3_key = model_data_url.split('s3://{}/'.format(BUCKET))[1]
    # get the part of the key that identifies the model within the model artifacts folder
    model_name_plus = s3_key[s3_key.find('model_artifacts') + len('model_artifacts') + 1:]
    # finally, get the unique model name (e.g., "NewYork_NY")
    model_name = re.findall('^(.*?)/', model_name_plus)[0]
    return s3_key, model_name 

In [28]:
# make a copy of the model artifacts from the original output of the training job to the place in
# s3 where the multi model endpoint will dynamically load individual models
def deploy_artifacts_to_mme(job_name):
    print("job_name :", job_name)
    response = sm_client.describe_training_job(TrainingJobName=job_name)
    source_s3_key, model_name = parse_model_artifacts(response['ModelArtifacts']['S3ModelArtifacts'])
    copy_source = {'Bucket': BUCKET, 'Key': source_s3_key}
    key = '{}/{}/{}/{}.tar.gz'.format(DATA_PREFIX, MULTI_MODEL_ARTIFACTS, model_name, model_name)
    
    print('Copying {} model\n   from: {}\n     to: {}...'.format(model_name, source_s3_key, key))
    s3_client.copy_object(Bucket=BUCKET, CopySource=copy_source, Key=key)


In [29]:
# First, clear out old versions of the model artifacts from previous runs of this notebook
s3_bucket = s3.Bucket(BUCKET)
full_input_prefix = '{}/multi_model_artifacts'.format(DATA_PREFIX)
print('Removing old model artifacts from {}'.format(full_input_prefix))
s3_bucket.objects.filter(Prefix=full_input_prefix + '/').delete()

Removing old model artifacts from DEMO_MME_LINEAR_LEARNER/multi_model_artifacts


[{'ResponseMetadata': {'RequestId': 'A6XRAVBHG9P64MJ7',
   'HostId': '5OWpvxHM5CKMheg3LTgXaVoiuOifGclHIWObej6Pb3fGE0vV+c8Jl2zA5XEVENCRx4khNj5fB88=',
   'HTTPStatusCode': 200,
   'HTTPHeaders': {'x-amz-id-2': '5OWpvxHM5CKMheg3LTgXaVoiuOifGclHIWObej6Pb3fGE0vV+c8Jl2zA5XEVENCRx4khNj5fB88=',
    'x-amz-request-id': 'A6XRAVBHG9P64MJ7',
    'date': 'Fri, 28 May 2021 21:03:00 GMT',
    'content-type': 'application/xml',
    'transfer-encoding': 'chunked',
    'server': 'AmazonS3',
    'connection': 'close'},
   'RetryAttempts': 0},
  'Deleted': [{'Key': 'DEMO_MME_LINEAR_LEARNER/multi_model_artifacts/Houston_TX/174_Houston_TX.tar.gz'},
   {'Key': 'DEMO_MME_LINEAR_LEARNER/multi_model_artifacts/Houston_TX/41_Houston_TX.tar.gz'},
   {'Key': 'DEMO_MME_LINEAR_LEARNER/multi_model_artifacts/Houston_TX/38_Houston_TX.tar.gz'},
   {'Key': 'DEMO_MME_LINEAR_LEARNER/multi_model_artifacts/Chicago_IL/Chicago_IL.tar.gz'},
   {'Key': 'DEMO_MME_LINEAR_LEARNER/multi_model_artifacts/NewYork_NY/NewYork_NY.tar.gz'},

In [30]:
## Deploy all but the last model trained to MME
## We will use the last model to show how to update an existing MME in Section 7
for job_name in training_jobs[:-1]:
    deploy_artifacts_to_mme(job_name)

job_name : linear-learner-2021-05-28-20-56-51-295
Copying NewYork_NY model
   from: DEMO_MME_LINEAR_LEARNER/model_artifacts/NewYork_NY/linear-learner-2021-05-28-20-56-51-295/output/model.tar.gz
     to: DEMO_MME_LINEAR_LEARNER/multi_model_artifacts/NewYork_NY/NewYork_NY.tar.gz...
job_name : linear-learner-2021-05-28-20-56-51-472
Copying LosAngeles_CA model
   from: DEMO_MME_LINEAR_LEARNER/model_artifacts/LosAngeles_CA/linear-learner-2021-05-28-20-56-51-472/output/model.tar.gz
     to: DEMO_MME_LINEAR_LEARNER/multi_model_artifacts/LosAngeles_CA/LosAngeles_CA.tar.gz...
job_name : linear-learner-2021-05-28-20-56-53-549
Copying Chicago_IL model
   from: DEMO_MME_LINEAR_LEARNER/model_artifacts/Chicago_IL/linear-learner-2021-05-28-20-56-53-549/output/model.tar.gz
     to: DEMO_MME_LINEAR_LEARNER/multi_model_artifacts/Chicago_IL/Chicago_IL.tar.gz...


In [31]:
MODEL_NAME = '{}-{}'.format(HOUSING_MODEL_NAME, strftime('%Y-%m-%d-%H-%M-%S', gmtime()))

_model_url  = 's3://{}/{}/{}/'.format(BUCKET, DATA_PREFIX, MULTI_MODEL_ARTIFACTS)

ll_multi_model = MultiDataModel(
        name=MODEL_NAME,
        model_data_prefix=_model_url,
        image_uri=container,
        role=role,
        sagemaker_session=sagemaker_session
    )

## Section 5 : Create an inference pipeline with sklearn model and MME linear learner model <a id='Create-inference-pipeline'></a>

Set up the inference pipeline using the Pipeline Model API.  This sets up a list of models in a single endpoint; In this example, we configure our pipeline model with the fitted Scikit-learn inference model and the fitted Linear Learner model.

In [32]:
from sagemaker.model import Model
from sagemaker.pipeline import PipelineModel
import boto3
from time import gmtime, strftime

timestamp_prefix = strftime("%Y-%m-%d-%H-%M-%S", gmtime())

scikit_learn_inference_model = sklearn_preprocessor.create_model()

model_name = '{}-{}'.format('inference-pipeline', timestamp_prefix)
endpoint_name = '{}-{}'.format('inference-pipeline-ep', timestamp_prefix)

sm_model = PipelineModel(
    name=model_name, 
    role=role, 
    sagemaker_session=sagemaker_session,
    models=[
        scikit_learn_inference_model, 
        ll_multi_model])

sm_model.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge', endpoint_name=endpoint_name)

-------------------!

## Section 6 :  Exercise the inference pipeline - Get predictions from  different  linear learner models. <a id='Exercise-inference-pipeline'></a>

In [33]:
#Create Predictor
from sagemaker.predictor import Predictor

csv_serializer = sagemaker.serializers.CSVSerializer()

predictor = Predictor(
    endpoint_name=endpoint_name,
    sagemaker_session=sagemaker_session,
    serializer=csv_serializer)

In [34]:
def predict_one_house_value(features, model_name, predictor_to_use):
    print('Using model {} to predict price of this house: {}'.format(model_name,
                                                                     features))
    body = ','.join(map(str, features)) + '\n'
    start_time = time.time()
     
    response = predictor_to_use.predict(features, target_model=model_name)
    
    response_json = json.loads(response)
        
    predicted_value = response_json['predictions'][0]['score']    
    
    duration = time.time() - start_time
    
    print('${:,.2f}, took {:,d} ms\n'.format(predicted_value, int(duration * 1000)))

In [35]:
for _ in range(10):
    model_name = LOCATIONS[np.random.randint(1, PARALLEL_TRAINING_JOBS - 1)]
    full_model_name = '{}/{}.tar.gz'.format(model_name,model_name)
    predict_one_house_value(gen_random_house()[:-1], full_model_name, predictor)

Using model LosAngeles_CA/LosAngeles_CA.tar.gz to predict price of this house: [1995, 3733.4944153545366, 2, 1.0, 1.37, 0, 'n', 'n']
$501,692.75, took 2,214 ms

Using model Chicago_IL/Chicago_IL.tar.gz to predict price of this house: [2000, 3093.065104045882, 5, 1.5, 0.79, 3, 'n', 'y']
$499,528.06, took 1,761 ms

Using model LosAngeles_CA/LosAngeles_CA.tar.gz to predict price of this house: [1994, 2389.7623356967097, 2, 2.5, 1.02, 0, 'n', 'n']
$312,219.31, took 49 ms

Using model LosAngeles_CA/LosAngeles_CA.tar.gz to predict price of this house: [1988, 2649.787223089696, 5, 2.5, 1.12, 0, 'y', 'n']
$373,254.16, took 45 ms

Using model LosAngeles_CA/LosAngeles_CA.tar.gz to predict price of this house: [1991, 2513.9827621539066, 3, 1.0, 1.22, 1, 'n', 'y']
$323,959.56, took 45 ms

Using model Chicago_IL/Chicago_IL.tar.gz to predict price of this house: [2017, 2717.655916259325, 4, 2.5, 1.09, 0, 'y', 'y']
$517,623.44, took 40 ms

Using model LosAngeles_CA/LosAngeles_CA.tar.gz to predict pri

## Section 7 - Add new model to the endpoint, simply by copying the model artifact to the S3 location
<a id='update-models'></a>

In [36]:
## Copy the last model
last_training_job=training_jobs[PARALLEL_TRAINING_JOBS-1]
deploy_artifacts_to_mme(last_training_job)

job_name : linear-learner-2021-05-28-20-56-58-082
Copying Houston_TX model
   from: DEMO_MME_LINEAR_LEARNER/model_artifacts/Houston_TX/linear-learner-2021-05-28-20-56-58-082/output/model.tar.gz
     to: DEMO_MME_LINEAR_LEARNER/multi_model_artifacts/Houston_TX/Houston_TX.tar.gz...


In [37]:
model_name = LOCATIONS[PARALLEL_TRAINING_JOBS-1]
full_model_name = '{}/{}.tar.gz'.format(model_name,model_name)
predict_one_house_value(gen_random_house()[:-1], full_model_name, predictor)

Using model Houston_TX/Houston_TX.tar.gz to predict price of this house: [1995, 5310.646499516446, 3, 2.5, 0.93, 1, 'y', 'n']
$793,874.81, took 1,657 ms



## Section 8 - Endpoint CloudWatch Metrics Analysis <a id='CW-metric-analysis'></a>

With MME, the models are dynamically loaded into the container’s memory of the instance hosting the endpoint when invoked.  Therefore, the model invocation may take longer when it is invoked for the first time. And after the model is already in the container’s memory, the subsequent invocations will be faster. If an instance memory utilization is high and a new model needs to be loaded then unused models are unloaded.  The unloaded models will remain in the instance’s storage volume and can be loaded into container’s memory later without being downloaded from the S3 bucket again.  If the instance’s storage volume if full, unused models are deleted from storage volume.    
Managing the loading/unloading of the models is completely handled by Amazon SageMaker behind the scenes without you having to take any specific actions.  However, it is important to understand this behavior because it has implications on the model invocation latency.

Amazon SageMaker provides CloudWatch metrics for multi-model endpoints so you can determine the endpoint usage and the cache hit rate and optimize your endpoint.  To analyze the endpoint and the container behavior, you will invoke multiple models in this order :

    a. Create 200 copies of the original model and save with different names.
    b. Starting with no models loaded into the container, Invoke the first 100 models
    c. Invoke the same 100 models again
    d. Invoke all 200 models

We use this order of invocations to observe the behavior of the CloudWatch metrics - LoadedModelCount, MemoryUtilization and ModelCacheHit.  You are encouraged to experiment with loading varying number of models to use the CloudWatch charts to help make ongoing decisions on the optimal choice of instance type, instance count, and number of models that a given endpoint should host.



In [38]:
# Make a copy of the model artifacts in S3 bucket with new names so we have multiple models to understand the latency behavior.
def copy_additional_artifacts_to_mme(num_copies):
    
    source_s3_model_key = '{}/{}/{}/{}.tar.gz'.format(DATA_PREFIX, MULTI_MODEL_ARTIFACTS, model_name, model_name)
    _copy_source = {'Bucket': BUCKET, 'Key': source_s3_model_key}
    for i in range(num_copies):
        new_model_name="{}_{}".format(i, model_name)
        dest_s3_model_key = '{}/{}/{}/{}.tar.gz'.format(DATA_PREFIX, MULTI_MODEL_ARTIFACTS, model_name, new_model_name)
        print('Copying {} model\n   from: {}\n     to: {}...'.format(model_name, source_s3_model_key, dest_s3_model_key))
        s3_client.copy_object(Bucket=BUCKET, CopySource=_copy_source, Key=dest_s3_model_key)

In [39]:
##Create 200 copies of the original model and save with different names.
copy_additional_artifacts_to_mme(200)

Copying Houston_TX model
   from: DEMO_MME_LINEAR_LEARNER/multi_model_artifacts/Houston_TX/Houston_TX.tar.gz
     to: DEMO_MME_LINEAR_LEARNER/multi_model_artifacts/Houston_TX/0_Houston_TX.tar.gz...
Copying Houston_TX model
   from: DEMO_MME_LINEAR_LEARNER/multi_model_artifacts/Houston_TX/Houston_TX.tar.gz
     to: DEMO_MME_LINEAR_LEARNER/multi_model_artifacts/Houston_TX/1_Houston_TX.tar.gz...
Copying Houston_TX model
   from: DEMO_MME_LINEAR_LEARNER/multi_model_artifacts/Houston_TX/Houston_TX.tar.gz
     to: DEMO_MME_LINEAR_LEARNER/multi_model_artifacts/Houston_TX/2_Houston_TX.tar.gz...
Copying Houston_TX model
   from: DEMO_MME_LINEAR_LEARNER/multi_model_artifacts/Houston_TX/Houston_TX.tar.gz
     to: DEMO_MME_LINEAR_LEARNER/multi_model_artifacts/Houston_TX/3_Houston_TX.tar.gz...
Copying Houston_TX model
   from: DEMO_MME_LINEAR_LEARNER/multi_model_artifacts/Houston_TX/Houston_TX.tar.gz
     to: DEMO_MME_LINEAR_LEARNER/multi_model_artifacts/Houston_TX/4_Houston_TX.tar.gz...
Copying Ho

In [40]:
##Invoke multiple models in a loop
def invoke_multiple_models_mme(model_range_low, model_range_high):
    for i in range(model_range_low, model_range_high):
        new_model_name="{}_{}".format(i, model_name)
        full_model_name = '{}/{}.tar.gz'.format(model_name, new_model_name)
        predict_one_house_value(gen_random_house()[:-1], full_model_name, predictor)


In [41]:
##Starting with no models loaded into the container
##Invoke the first 100 models
invoke_multiple_models_mme(0, 100)

Using model Houston_TX/0_Houston_TX.tar.gz to predict price of this house: [1981, 4000.6605137886877, 5, 2.5, 0.42, 3, 'y', 'n']
$564,194.00, took 1,686 ms

Using model Houston_TX/1_Houston_TX.tar.gz to predict price of this house: [2009, 2807.5788173200076, 4, 1.5, 1.6, 3, 'y', 'y']
$533,546.00, took 1,600 ms

Using model Houston_TX/2_Houston_TX.tar.gz to predict price of this house: [2003, 3393.5713949484784, 2, 1.5, 1.41, 3, 'n', 'y']
$538,403.00, took 1,623 ms

Using model Houston_TX/3_Houston_TX.tar.gz to predict price of this house: [1974, 4036.868425355425, 2, 2.0, 1.04, 3, 'n', 'y']
$481,763.56, took 1,625 ms

Using model Houston_TX/4_Houston_TX.tar.gz to predict price of this house: [1984, 3581.4609510869664, 5, 1.0, 0.93, 3, 'n', 'y']
$477,466.38, took 1,605 ms

Using model Houston_TX/5_Houston_TX.tar.gz to predict price of this house: [1986, 2919.4720854301727, 4, 1.5, 1.03, 1, 'n', 'n']
$359,295.25, took 1,592 ms

Using model Houston_TX/6_Houston_TX.tar.gz to predict price 

In [42]:
##Invoke the same 100 models again
invoke_multiple_models_mme(0, 100)

Using model Houston_TX/0_Houston_TX.tar.gz to predict price of this house: [1993, 2059.212348811203, 4, 2.5, 0.83, 3, 'n', 'n']
$312,405.47, took 1,645 ms

Using model Houston_TX/1_Houston_TX.tar.gz to predict price of this house: [1973, 3274.6551252758586, 4, 1.0, 0.89, 0, 'y', 'n']
$339,918.25, took 1,654 ms

Using model Houston_TX/2_Houston_TX.tar.gz to predict price of this house: [1997, 2677.4526874919975, 3, 1.5, 0.81, 2, 'y', 'y']
$404,714.91, took 1,646 ms

Using model Houston_TX/3_Houston_TX.tar.gz to predict price of this house: [1981, 1454.9114542448885, 4, 1.5, 1.14, 3, 'y', 'n']
$172,242.12, took 1,608 ms

Using model Houston_TX/4_Houston_TX.tar.gz to predict price of this house: [2001, 3487.4531104647854, 2, 2.5, 1.44, 0, 'n', 'y']
$515,330.12, took 1,695 ms

Using model Houston_TX/5_Houston_TX.tar.gz to predict price of this house: [2016, 2710.329286374311, 3, 1.5, 1.18, 1, 'n', 'y']
$479,196.41, took 1,647 ms

Using model Houston_TX/6_Houston_TX.tar.gz to predict price 

In [43]:
##This time invoke all 200 models to observe behavior
invoke_multiple_models_mme(0, 200)

Using model Houston_TX/0_Houston_TX.tar.gz to predict price of this house: [2018, 3725.099880583392, 5, 2.5, 1.61, 2, 'y', 'n']
$732,573.88, took 1,628 ms

Using model Houston_TX/1_Houston_TX.tar.gz to predict price of this house: [1995, 2335.421903559468, 2, 3.0, 1.14, 1, 'y', 'y']
$353,438.97, took 1,591 ms

Using model Houston_TX/2_Houston_TX.tar.gz to predict price of this house: [1990, 3374.6527811756982, 2, 1.0, 1.28, 0, 'y', 'n']
$432,486.84, took 1,601 ms

Using model Houston_TX/3_Houston_TX.tar.gz to predict price of this house: [2001, 2916.743932407746, 5, 3.0, 1.06, 1, 'n', 'y']
$478,079.44, took 1,598 ms

Using model Houston_TX/4_Houston_TX.tar.gz to predict price of this house: [1977, 1904.4289231718374, 3, 2.5, 0.89, 1, 'n', 'n']
$164,743.47, took 1,623 ms

Using model Houston_TX/5_Houston_TX.tar.gz to predict price of this house: [1988, 3470.80307702986, 2, 1.0, 0.89, 1, 'n', 'n']
$417,433.56, took 1,575 ms

Using model Houston_TX/6_Houston_TX.tar.gz to predict price of 

#### CloudWatch charts for LoadedModelCount,MemoryUtilization and ModelCacheHit metrics will be similar to charts below.

![](cw_charts/ModelCountMemUtilization.png)

“LoadedModelCount” continuously increases, as more models are invoked, till it levels off at 121.  “MemoryUtilization” of the container also increased correspondingly to around 79%.  This shows that the instance chosen to host the endpoint, could only maintain 121 models in memory, when 200 model invocations are made.  

![](cw_charts/ModelCountMemUtilizationCacheHit.png)

As the number of models loaded to the container memory increase, the ModelCacheHit improves.  When the same 100 models are invoked the second time, the ModelCacheHit reaches 1.  When new models, not yet loaded are invoked the ModelCacheHit decreases again. 

## Section 9 - Explore granular access to the target models of MME <a id='Finegrain-control-invoke-models'></a>

If the role attached to this notebook instance allows invoking SageMaker endpoints, it is able to invoke all models hosted on the MME.  Using IAM conditional keys, you can restrict this model invocation access to specific models.  To explore this, you will create a new IAM role and IAM policy with conditional key to restrict access to a single model.  Assume this new role and verify that only a single target model can be invoked.

Note that to execute this section, the role attached to the notebook instance should allow the following actions :
    "iam:CreateRole",
    "iam:CreatePolicy",
    "iam:AttachRolePolicy",
    "iam:UpdateAssumeRolePolicy"
    
If this is not the case, please work with the Administrator of this AWS account to ensure this.  

In [44]:
iam_client = boto3.client('iam')

In [45]:
#Create a new role that can be assumed by this notebook.  The roles should allow access to only a single model.

path='/'

role_name="{}{}".format('allow_invoke_ny_model_role', strftime('%Y-%m-%d-%H-%M-%S', gmtime()))
description='Role that allows invoking a single model'

action_string = "sts:AssumeRole"
    
trust_policy={
  "Version": "2012-10-17",
  "Statement": [
    {
      "Sid": "statement1",
      "Effect": "Allow",
      "Principal": {
        "AWS": role
      },
      "Action": "sts:AssumeRole"
    }
  ]
}

In [46]:
response = iam_client.create_role(
    Path=path,
    RoleName=role_name,
    AssumeRolePolicyDocument=json.dumps(trust_policy),
    Description=description,
    MaxSessionDuration=3600
)

In [47]:
role_arn=response['Role']['Arn']
print("Role arn is :", role_arn)

Role arn is : arn:aws:iam::688520471316:role/allow_invoke_ny_model_role2021-05-28-21-23-36


In [48]:
endpoint_resource_arn = "arn:aws:sagemaker:{}:{}:endpoint/{}".format(REGION, ACCOUNT_ID, endpoint_name)
print("Endpoint arn is :", endpoint_resource_arn)

Endpoint arn is : arn:aws:sagemaker:us-west-2:688520471316:endpoint/inference-pipeline-ep-2021-05-28-21-03-00


In [49]:
##Create the IAM policy with the IAM condition key
policy_name = "{}{}".format('allow_invoke_ny_model_policy', strftime('%Y-%m-%d-%H-%M-%S', gmtime()))
managed_policy = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "SageMakerAccess",
            "Action": "sagemaker:InvokeEndpoint",
            "Effect": "Allow",
            "Resource":endpoint_resource_arn,
            "Condition": {
                "StringLike": {
                    "sagemaker:TargetModel": ["NewYork_NY/*"]
                }
            }
        }
    ]
}

response = iam_client.create_policy(
  PolicyName=policy_name,
  PolicyDocument=json.dumps(managed_policy)
)

In [50]:
policy_arn=response['Policy']['Arn']

In [51]:
##Attach policy to role
iam_client.attach_role_policy(
    PolicyArn=policy_arn,
    RoleName=role_name
)

{'ResponseMetadata': {'RequestId': '4619d603-f251-495e-9e1d-15c09fed6a92',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '4619d603-f251-495e-9e1d-15c09fed6a92',
   'content-type': 'text/xml',
   'content-length': '212',
   'date': 'Fri, 28 May 2021 21:23:36 GMT'},
  'RetryAttempts': 0}}

In [52]:
## Invoke with the role that has access to only NY model
sts_connection = boto3.client('sts')
assumed_role_limited_access = sts_connection.assume_role(
    RoleArn=role_arn,
    RoleSessionName="MME_Invoke_NY_Model"
)
assumed_role_limited_access['AssumedRoleUser']['Arn']


ClientError: An error occurred (AccessDenied) when calling the AssumeRole operation: User: arn:aws:sts::688520471316:assumed-role/hongshan-sagemaker-experiment/i-01b049c9062e751b4 is not authorized to perform: sts:AssumeRole on resource: arn:aws:iam::688520471316:role/allow_invoke_ny_model_role2021-05-28-21-23-36

In [ ]:
trust_policy={
  "Version": "2012-10-17",
  "Statement": [
    {
      "Sid": "statement1",
      "Effect": "Allow",
      "Principal": {
        "AWS": role
      },
      "Action": "sts:AssumeRole"
    },
    {
      "Sid": "statement2",
      "Effect": "Allow",
      "Principal": {
          "AWS": assumed_role_limited_access['AssumedRoleUser']['Arn']
      },
      "Action": "sts:AssumeRole"
    }  
  ]
}

In [ ]:
iam_client.update_assume_role_policy(
    RoleName=role_name,
    PolicyDocument=json.dumps(trust_policy)
)

In [ ]:
ACCESS_KEY = assumed_role_limited_access['Credentials']['AccessKeyId']
SECRET_KEY = assumed_role_limited_access['Credentials']['SecretAccessKey']
SESSION_TOKEN = assumed_role_limited_access['Credentials']['SessionToken']

runtime_sm_client_with_assumed_role = boto3.client(
    service_name='sagemaker-runtime', 
    aws_access_key_id=ACCESS_KEY,
    aws_secret_access_key=SECRET_KEY,
    aws_session_token=SESSION_TOKEN,
)

In [ ]:
 sagemakerSessionAssumedRole = sagemaker.Session(sagemaker_runtime_client=runtime_sm_client_with_assumed_role)

In [ ]:
predictorAssumedRole = Predictor(
    endpoint_name=endpoint_name,
    sagemaker_session=sagemakerSessionAssumedRole,
    serializer=csv_serializer)

In [ ]:
full_model_name = 'NewYork_NY/NewYork_NY.tar.gz'
predict_one_house_value(gen_random_house()[:-1], full_model_name,predictorAssumedRole)

In [ ]:
##This should fail with "AccessDeniedException" since the assumed role does not have access to Chicago model
full_model_name = 'Chicago_IL/Chicago_IL.tar.gz'
predict_one_house_value(gen_random_house()[:-1], full_model_name,predictorAssumedRole)

## Clean up<a id='CleanUp'></a>
Clean up the endpoint to avoid unneccessary costs.



In [ ]:
#Delete the endpoint and underlying model
predictor.delete_model() 
predictor.delete_endpoint()
for t in preprocessor_transformers:
    t.delete_model()

In [ ]:
#Delete the IAM Role
iam_client.detach_role_policy(
    PolicyArn=policy_arn,
    RoleName=role_name
)
iam_client.delete_role(RoleName=role_name)

In [ ]:
#Delete the IAM Policy
iam_client.delete_policy(PolicyArn=policy_arn)